In [79]:
import numpy as np
xs = [0, 0.24, 0.55, 0.73, 1]
def f(x):
    return (x * np.tan(x))/(1+np.tan(x**2))
ys = [round(f(x), 10) for x in xs]


In [18]:
def Li(xs, ys, i):
    ans = 1
    for j in range(len(xs)):
        if j != i:
            ans *= xs[i] - xs[j]
    return round(ans * ys[i], 10)
Li(xs, ys, 4)

0.0562331254

In [26]:
def calc_nevil(Q, xs, i,j, x):
    print(f"(x-x_{i-j})Q_"+"{"+f"{i},{j-1}" + "}" + f" - (x- x_{i})Q_"+"{"+f"{i-1},{j-1}" + "}", f"x_{i}-x_{i-j}")
    if j == 1:
        print(f"(x-{xs[i-j]}){Q[j-1][i]} - (x-{xs[i]}){Q[j-1][i-1]}", f"{xs[i]}-{xs[i-j]}")
    else:
        print(f"(x-{xs[i-j]})Q_"+"{"+f"{i},{j-1}" + "}" + f"- (x-{xs[i]})Q_"+"{"+f"{i-1},{j-1}" + "}", f"{xs[i]}-{xs[i-j]}")
#     print(f"({x}-{xs[i-j]}){Q[j-1][i]} - ({x}-{xs[i]}){Q[j-1][i-1]}", f"({xs[i]}-{xs[i-j]})")
    return round((((x-xs[i-j])*Q[j-1][i]) - ((x-xs[i])*Q[j-1][i-1]))/(xs[i]-xs[i-j]), 10)

def partial_nevil(Q, xs, i, j, x):
    return ((x-xs[i])*Q[i+1][j] - (x-xs[j])*Q[i][j-1])/(xs[j] - xs[i])

def nevil(ys, xs, x):
    Q = [[0]*len(xs) for _ in xs]
    for i in range(len(xs)):
        Q[i][i] = ys[i]
    for i in range(1, len(xs)):
        for j in range(len(xs) - i):
            Q[j][j+i] = partial_nevil(Q, xs, j, j+i, x)
    return Q

xs = [1.0, 1.3, 1.6, 1.9, 2.2]
ys = [0.7651977, 0.6200860, 0.4554022, 0.2818186, 0.1103623]
Q = nevil(ys, xs, 1.5)

In [30]:
def divideddiff(xs, ys):
    F = [ys]
#     print(f"{ys}")
    for i in range(1, len(xs)):
        fp = []
        for j in range(len(xs) - i):
            a = (F[i-1][j+1] - F[i-1][j])
            b = (xs[i+j] - xs[j])
            ans = round(a/b, 10)
            fp.append(ans)
#             print("$F_{" + f"{i}, {j}" + "} =" + "\dfrac{ F_{" + f"{i-1},{j+1}" + "} - " + "F_{" + f"{i-1}, {j}" + "}}{" + f"x_{i+j} - x_{j}" + "} = " + "\dfrac{" + f"{F[i-1][j+1]} - {F[i-1][j]}" + "}{" + f"{xs[i+j]} - {xs[j]}"+ "} = " + f"{ans}$\\\\")
        F.append(fp)
#     print("P(x) = ", end="")
#     for i in range(len(xs)):
#         print(f"{F[i][0]} \\times ", end="")
#         for j in range(i+1):
#             print(f"(x-{xs[j]})", end="")
#         print(" + ", end="")
    return F

for x in divideddiff(xs, ys):
    print(x)

[0.7651977, 0.620086, 0.4554022, 0.2818186, 0.1103623]
[-0.4837056667, -0.548946, -0.578612, -0.571521]
[-0.1087338888, -0.0494433333, 0.0118183333]
[0.065878395, 0.0680685184]
[0.0018251028]


In [21]:
import numpy as np
import scipy as sp
import math

def cubic_spline(xs, ys):
    n = len(xs)
    b = [0]*(n+1)
    c = [0]*(n+1)
    d = [0]*(n+1)
    h = [xs[i+1]-xs[i] for i in range(n-1)]
    a = [0] + [3*(ys[i+1]-ys[i])/h[i] - 3*(ys[i]-ys[i-1])/h[i-1] for i in range(1, n-1)]
    l = [0]*(n+1)
    mu = [0]*(n+1)
    z = [0]*(n+1)
    l[0] = 1
    for i in range(1, n-1):
        l[i] = 2*(xs[i+1]-xs[i-1])-h[i-1]*mu[i-1]
        mu[i] = h[i]/l[i]
        z[i] = (a[i]-h[i-1]*z[i-1])/l[i]
    l[n] = 1
    for i in range(n-2, -1, -1):
        c[i] = z[i] - mu[i]*c[i+1]
        b[i] = (ys[i+1]-ys[i])/h[i] - h[i]*(c[i+1]+2*c[i])/3
        d[i] = (c[i+1]-c[i])/(3*h[i])
    return b, c, d


xs = range(4)
ys = [math.exp(i) for i in xs]
cubic_spline(xs, ys)

([1.465997614174723, 2.222850257027689, 8.809769654506473, 0, 0],
 [0.0, 0.756852642852966, 5.830066754625817, 0, 0],
 [0.252284214284322, 1.6910713705909506, -1.9433555848752724, 0, 0])